In [8]:
from pathlib import Path
import pickle

import open3d as o3
import numpy as np
from sklearn.neighbors import KDTree
from typing import *

from pointcloud.config import DATA_PATH
from pointcloud.utils.io import read_ply_file
from pointcloud.processors.sensat.plot import draw_pointcloud, draw_segmented_pointcloud

LABELS = {
    0: "Ground",
    1: "High Vegetation",
    2: "Buildings",
    3: "Walls",
    4: "Bridge",
    5: "Parking",
    6: "Rail",
    7: "traffic Roads",
    8: "Street Furniture",
    9: "Cars",
    10: "Footpath",
    11: "Bikes",
    12: "Water",
}

pointcloud_path = Path(
    DATA_PATH / "sensat_urban/grid_0.2/birmingham_block_11_sample.ply"
)
kd_tree_path = Path(DATA_PATH / "sensat_urban/grid_0.2/birmingham_block_11_KDTree.pkl")
proj_path = Path(DATA_PATH / "sensat_urban/grid_0.2/birmingham_block_11_proj.pkl")

assert pointcloud_path.exists(), "PointCloud file not found."
assert kd_tree_path.exists(), "KDTree file not found."
assert proj_path.exists(), "Proj file not found."

In [11]:
points, colors, labels = read_ply_file(pointcloud_path)

In [2]:
LABEL_COLORS = [
    [85, 107, 47],  # ground -> OliveDrab
    [0, 255, 0],  # tree -> Green
    [255, 165, 0],  # building -> orange
    [41, 49, 101],  # Walls ->  darkblue
    [0, 0, 0],  # Bridge -> black
    [0, 0, 255],  # parking -> blue
    [255, 0, 255],  # rail -> Magenta
    [200, 200, 200],  # traffic Roads ->  grey
    [89, 47, 95],  # Street Furniture  ->  DimGray
    [255, 0, 0],  # cars -> red
    [255, 255, 0],  # Footpath  ->  deeppink
    [0, 255, 255],  # bikes -> cyan
    [0, 191, 255],  # water ->  skyblue
]

In [16]:
# Path to pointcloud file that has not been sub-sampled
pointcloud_path = Path(
    "/home/macdonaldezra/Desktop/code/grid_0.200/birmingham_block_11.ply"
)
assert pointcloud_path.exists(), "PointCloud file not found."

In [17]:
points, colors, labels = read_ply_file(pointcloud_path)

In [11]:
draw_pointcloud(points, colors)

In [20]:
_ = draw_segmented_pointcloud(points, labels)

In [21]:
# Read in tree file
with open(kd_tree_path, "rb") as kd_tree_file:
    tree = pickle.load(kd_tree_file)

In [28]:
def shuffle_indices(arr: np.ndarray) -> np.ndarray:
    """
    Randomly shuffle an input array's indices and returned array with shuffled index.
    """
    indices = np.arange(len(arr))
    np.random.shuffle(indices)

    return arr[indices]


def compute_distances(arr: np.ndarray, point: float) -> np.ndarray:
    """
    Return the absolute normalized distance between a set of points and a given point.
    """
    distances = np.sum((np.square(arr - point)).astype(np.float32), axis=-1)
    return np.square((1 - distances) / np.max(distances))


def resize_pointcloud_inputs(
    points: np.ndarray,
    colors: np.ndarray,
    labels: np.ndarray,
    indices: np.ndarray,
    size: int,
) -> List[np.ndarray]:
    """
    Resize PointCloud input array to match the size parameter.

    This functionality was taken from the original Sensat implementation and has been created to better understand,
    how their research team is modifying input data
    """
    num_inputs = len(points)
    duplicate = np.random.choice(num_inputs, size - num_inputs)
    point_duplicates = points[duplicate, ...]
    points_resized = np.concatenate([points, point_duplicates], 0)

    color_duplicates = colors[duplicate, ...]
    colors_resized = np.concatenate([colors, color_duplicates], 0)

    duplicate_indices = list(range(size)) + list(duplicate)
    resize_index = indices[duplicate_indices]
    label_duplicates = labels[duplicate, ...]
    labels_resized = np.concatenate([labels, label_duplicates], 0)

    return [points_resized, colors_resized, labels_resized, resize_index]

In [55]:
MODEL_INPUT_SIZE = 40960
# How Sensat repos handle generating a training batch for a model

possibility = np.random.rand(tree.data.shape[0]) * 1e-3
min_possibility = float(np.min(possibility))

# Choose the point with the minimum of possibility as query points
point_index = np.argmin(possibility)
# Get points from the tree structure
points = np.array(tree.data, copy=False)
center_point = points[point_index, :].reshape(1, -1)

# Add noise to the center point
noise = np.random.normal(scale=3.5 / 10, size=center_point.shape)
chosen_point = center_point + noise.astype(center_point.dtype)

# If PointCloud is smaller than model input size, then
if len(points) < MODEL_INPUT_SIZE:  # Chosen number of points
    queried_index = tree.query(chosen_point, k=len(points))[1][0]
else:
    queried_index = tree.query(chosen_point, k=MODEL_INPUT_SIZE)[1][0]

queried_index = shuffle_indices(queried_index)
queried_points = points[queried_index]
queried_points -= chosen_point
queried_colors = colors[queried_index]
queried_labels = labels[queried_index]

# possibility[queried_index] += compute_distances(queried_points, chosen_point)
# min_possibility = float(np.min(possibility))

if len(points) < MODEL_INPUT_SIZE:
    (
        queried_points,
        queried_colors,
        queried_labels,
        queried_index,
    ) = resize_pointcloud_inputs(
        queried_points, queried_colors, queried_labels, queried_index, MODEL_INPUT_SIZE
    )

In [56]:
draw_pointcloud(queried_points, queried_colors)

In [57]:
_ = draw_segmented_pointcloud(queried_points, queried_labels)

In [53]:
np.min(possibility)

7.262276605413121e-09

In [54]:
np.max(possibility)

0.0009999869840940263

In [51]:
min_possibility

7.955882428412053e-09